In [1]:
import pandas as pd
import numpy as np
import regex as re
from sklearn.model_selection import train_test_split
from collections import Counter
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pack_padded_sequence,pad_packed_sequence
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
import seaborn as sns

In [2]:
train_file = pd.read_csv('../data/augmented_training_file/final_train_file2.csv')
final_test_file = pd.read_csv('../data/final_test_file.csv')


In [3]:
data_per_class=Counter(train_file['label'])
print("Number of rows grouped by label are:")
for i,label in enumerate(data_per_class):
    print(f"{label} class rows: {data_per_class[label]} ({data_per_class[label] / train_file.shape[0]*100:.3f})% of total train data")


Number of rows grouped by label are:
0 class rows: 175598 (40.240)% of total train data
2 class rows: 112302 (25.735)% of total train data
1 class rows: 148476 (34.025)% of total train data


In [4]:
from sklearn.utils import class_weight  # .compute_class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_file['label']), train_file['label'])
class_weights = torch.tensor(class_weights, dtype=torch.float)
print(f"Class weights to be used for CrossEntropyLoss (increases loss to classes with less data to balance the learning)\n {class_weights}")


Class weights to be used for CrossEntropyLoss (increases loss to classes with less data to balance the learning)
 tensor([0.8284, 0.9797, 1.2952])


C:\ProgramData\Anaconda3\envs\CVAS4\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass classes=[0 1 2], y=0         0
1         0
2         0
3         0
4         0
         ..
436371    1
436372    1
436373    1
436374    1
436375    1
Name: label, Length: 436376, dtype: int64 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [5]:
test_split_size=0.3 # percentage of test split
train_split,test_split=train_test_split(train_file, test_size=test_split_size,stratify=train_file['label'],random_state=9)


In [6]:
# Checking if cuda supported GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [7]:
# Creating customised Image dataset to be used for dataloader
class CustomDataset(Dataset):
    def __init__(self, data_frame):
        self.data = data_frame['encoded_titles_combined']
        self.label = data_frame['label']
        self.length=len(data_frame)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        text_data = self.data[idx]
        text_data = np.array(text_data[1:-1].split(), dtype=int)
        text_data = torch.tensor(text_data)
        # print(type(self.label[idx]), label_encoding[self.label[idx]])
        label = torch.tensor(self.label[idx]).long()
        data=(text_data,label)
        return data
    

In [8]:
train_split_dataset=CustomDataset(train_split.reset_index())
test_split_dataset=CustomDataset(test_split.reset_index())

In [10]:
batch = 100
loaders = {
    'train': DataLoader(train_split_dataset, batch_size=batch, shuffle=True, num_workers=0),
    'test': DataLoader(test_split_dataset, batch_size=batch, shuffle=True, num_workers=0),
}

In [11]:
# Creating an LSTM net
class LSTM_simple(nn.Module):
    def __init__(self,vocab_size,embed_dimension,lstm_units,hidden_dimension):
        super().__init__()
        self.embeddings=nn.Embedding(vocab_size,embed_dimension)
        self.lstm_layer1= nn.LSTM(embed_dimension,lstm_units,hidden_dimension,bidirectional=True,batch_first=True)
        self.full_layer1 = nn.Linear(2*hidden_dimension,3)

    def forward(self, text):
        text=text.to(device=device)
        embedded_text=self.embeddings(text)
        lstm_out, (ht, ct) = self.lstm_layer1(embedded_text)
        ht=torch.cat((ht[-2, :, :], ht[-1, :, :]), dim=1)
        ht=self.full_layer1(ht)
        return ht


LSTM_simple_model = LSTM_simple(49491, 256, 128, 128).to(device=device) # 49491 is vocab len
optimizer=optim.SGD(LSTM_simple_model.parameters(),lr=0.01,momentum=0.9)
class_weights=class_weights.to(device=device)
loss_func=nn.CrossEntropyLoss(weight=class_weights)

In [ ]:
# return the accuracy of the test_split
def validation(model):
    # Test the model
    model.eval()
    with torch.no_grad():
        for sentences, labels in loaders['test']:
            sentences.to(device=device)
            labels=labels.to(device=device)
            test_output = model(sentences)
            loss = loss_func(test_output, labels)
            pred_y = torch.argmax(test_output, 1).data.squeeze()
            accuracy = (pred_y == labels).sum().item() / float(labels.size(0))
        return accuracy,loss
# validation()


Test Accuracy : 0.3333


In [ ]:
num_epochs = 1
training_loss_list = []
train_accuracy_list = []
validation_loss_list = []
validation_accuracy_list = []
def train(num_epochs, LSTM_simple_model, loaders):
    LSTM_simple_model.train()
    # Train the model
    total_step = len(loaders['train'])

    for epoch in range(num_epochs):
        
        for i, (data, labels) in enumerate(loaders['train']):
            training_loss = []
            train_accuracy = []
            validation_loss = []
            validation_accuracy = []
            data.to(device=device)
            labels.to(device=device)
            step_acc = 0.0
            # gives batch data, normalize x when iterate train_loader
            b_x = Variable(data)   # batch x
            b_y = Variable(labels)   # batch y
            b_x = b_x.to(device=device)
            b_y = b_y.to(device=device)

            # clear gradients for this training step
            optimizer.zero_grad()
            output = LSTM_simple_model(b_x)  # predicted output from the net
            pred_y = torch.argmax(output, 1).data.squeeze()

            step_acc = (pred_y == b_y).sum().item() / float(labels.size(0))
            #calc cross entropy loss
            loss = loss_func(output, b_y)
            train_accuracy.append(step_acc)
            training_loss.append(loss.item())
            val_acc, val_loss = validation(LSTM_simple_model)
            validation_accuracy.append(val_acc)
            validation_loss.append(val_loss)
            # backpropagation, compute gradients
            loss.backward()     # apply gradients
            optimizer.step()
            
            if(i//50==0):
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, acc: {:.4f}' .format(
                    epoch + 1, num_epochs, i + 1, total_step, loss.item(), step_acc))

train(num_epochs, LSTM_simple_model, loaders)


In [ ]:
torch.save(LSTM_simple_model.state_dict(),'./lstm model.pt')